In [4]:
"""
EOD HD fundamentals builder — v7
Outputs the columns:
• MACD (12,26,9)   ← raw MACD
• MACD Signal      ← 9-day EMA of MACD
• ATH % Chg, Recent-High % Chg, RSI-1 yr, Fair-Value metrics, etc.
Ticker is printed without the “.US” suffix.
"""

import datetime as _dt
import requests  as _rq
import pandas    as _pd

API_TOKEN = "67ffece4b2ae08.94077168"
BASE_URL  = "https://eodhd.com/api"
HEADERS   = {"User-Agent": "EOD-Fundamentals-Client/7.0"}


def _get_json(url: str):
    r = _rq.get(url, headers=HEADERS, timeout=30)
    r.raise_for_status()
    return r.json()


def _pct_from_high(curr, high):
    if curr is None or high in (None, 0):
        return None
    return (curr - high) / high


def _pct_to_value(curr, val):
    """(FairValue – CurrentPrice) / CurrentPrice"""
    if curr in (None, 0) or val is None:
        return None
    return (val - curr) / curr


def _strip_suffix(ticker: str) -> str:
    return ticker.rsplit(".", 1)[0]


# ---------- fundamental helpers -----------------------------------------------
def _annual_eps_dict(fund: dict) -> dict[int, float]:
    return {
        int(k[:4]): rec["epsActual"]
        for k, rec in fund.get("Earnings", {}).get("Annual", {}).items()
        if rec.get("epsActual") is not None
    }


def _sum_quarterly_eps(hist: list[dict]) -> dict[int, float]:
    eps_q = {}
    for rec in hist:
        eps = rec.get("epsActual")
        d   = _pd.to_datetime(rec.get("date") or rec.get("reportDate"), errors="coerce")
        if _pd.notna(d) and eps is not None:
            eps_q.setdefault(d.year, []).append(eps)
    return {y: sum(v) for y, v in eps_q.items() if len(v) == 4 and sum(v) != 0}


def _five_year_pe_series(df_price: _pd.DataFrame, eps_year: dict[int, float]) -> list[float]:
    pe = []
    for y in sorted(eps_year)[-5:]:
        yr_px = df_price[df_price["date"].dt.year == y]
        if not yr_px.empty and (eps := eps_year[y]) != 0:
            pe.append(yr_px.iloc[-1]["adjusted_close"] / eps)
    return pe


def CurrentPrice(ticker):
    url = f"{BASE_URL}/real-time/{ticker}?api_token={API_TOKEN}&fmt=json"
    d = _get_json(url)
    return d.get("close") or d.get("price") or d.get("lastPrice")


def ATH(ticker):
    today = _dt.date.today().strftime("%Y-%m-%d")
    url   = f"{BASE_URL}/eod/{ticker}?api_token={API_TOKEN}&from=1900-01-01&to={today}&adjusted=1&fmt=json"
    df    = _pd.DataFrame(_get_json(url))
    return None if df.empty else df["adjusted_close"].max()


# ---------- technical-indicator functions ---------------------------------------
def _latest_macd(ticker):
    """Return only the latest raw MACD value."""
    today  = _dt.date.today()
    url = (
        f"{BASE_URL}/technical/{ticker}?order=d"
        f"&from={today.replace(year=today.year-2)}&to={today}"
        f"&function=macd&fast_period=12&slow_period=26&signal_period=9"
        f"&api_token={API_TOKEN}&fmt=json"
    )
    try:
        dat = _get_json(url)
        return dat[0].get("macd") if dat else None
    except Exception:
        return None


def _latest_macd_signal(ticker):
    """Return only the latest 9-EMA of MACD (signal line)."""
    today  = _dt.date.today()
    url = (
        f"{BASE_URL}/technical/{ticker}?order=d"
        f"&from={today.replace(year=today.year-2)}&to={today}"
        f"&function=macd&fast_period=12&slow_period=26&signal_period=9"
        f"&api_token={API_TOKEN}&fmt=json"
    )
    try:
        dat = _get_json(url)
        return dat[0].get("signal") if dat else None
    except Exception:
        return None


def _latest_rsi_1y(ticker):
    today  = _dt.date.today()
    url = (
        f"{BASE_URL}/technical/{ticker}?order=d"
        f"&from={today.replace(year=today.year-2)}&to={today}"
        f"&function=rsi&period=252"
        f"&api_token={API_TOKEN}&fmt=json"
    )
    try:
        dat = _get_json(url)
        return dat[0]["rsi"] if dat else None
    except Exception:
        return None


def Fundamentals(ticker: str) -> _pd.DataFrame:
    fund = _get_json(f"{BASE_URL}/fundamentals/{ticker}?api_token={API_TOKEN}&fmt=json")

    today_dt = _dt.datetime.now()
    hist_url = (
        f"{BASE_URL}/eod/{ticker}?api_token={API_TOKEN}"
        f"&from={(today_dt.replace(year=today_dt.year-6)).strftime('%Y-%m-%d')}"
        f"&to={today_dt.strftime('%Y-%m-%d')}&adjusted=1&fmt=json"
    )
    dfp = _pd.DataFrame(_get_json(hist_url))
    if dfp.empty:
        dfp = _pd.DataFrame(columns=["date", "adjusted_close"])
    dfp["date"] = _pd.to_datetime(dfp["date"])

    # -- highs & % changes -----------------------------------------------------
    curr_price     = CurrentPrice(ticker)
    all_time_high  = ATH(ticker)
    recent_high    = dfp.loc[dfp["date"] > _pd.Timestamp(today_dt) - _pd.Timedelta(days=180),
                             "adjusted_close"].max(skipna=True)
    high_5y        = dfp["adjusted_close"].max(skipna=True)

    # -- earnings & P/E --------------------------------------------------------
    hi        = fund.get("Highlights", {})
    eps_ttm   = hi.get("DilutedEpsTTM")
    pe_api    = hi.get("PERatio")

    hist_raw  = fund.get("Earnings", {}).get("History", {})
    earnings  = list(hist_raw.values()) if isinstance(hist_raw, dict) else (hist_raw or [])
    eps_year  = _annual_eps_dict(fund) or _sum_quarterly_eps(earnings)
    pe_series = _five_year_pe_series(dfp, eps_year)
    avg_pe5y  = sum(pe_series) / len(pe_series) if len(pe_series) >= 2 else None

    if pe_api is None and eps_ttm and eps_ttm != 0:
        last_px = dfp.iloc[-1]["adjusted_close"] if not dfp.empty else curr_price
        pe_api  = last_px / eps_ttm

    eps_est   = hi.get("EPSEstimateNextYear")
    fair_val  = avg_pe5y * eps_ttm if avg_pe5y and eps_ttm else None
    fair_valF = avg_pe5y * eps_est if avg_pe5y and eps_est else None

    # -- technicals ------------------------------------------------------------
    macd_raw    = _latest_macd(ticker)
    macd_signal = _latest_macd_signal(ticker)
    rsi_1y      = _latest_rsi_1y(ticker)

    row = {
        "Ticker":             _strip_suffix(ticker),
        "Current Price":      curr_price,
        "All-Time High":      all_time_high,
        "ATH %Chg":           _pct_from_high(curr_price, all_time_high),
        "6-Month High":       recent_high,
        "Recent-High %Chg":   _pct_from_high(curr_price, recent_high),
        "5-Year High":        high_5y,
        "Current P/E":        pe_api,
        "Avg 5-Year P/E":     avg_pe5y,
        "Fair Value (TTM)":   fair_val,
        "Fair Value %Chg":    _pct_to_value(curr_price, fair_val),
        "Avg P/E × Fwd EPS":  fair_valF,
        "MACD (12,26,9)":     macd_raw,       
        "MACD Signal":        macd_signal,    
        "RSI (1 yr)":         rsi_1y,
    }
    return _pd.DataFrame([row])


def FundamentalsBuilder(tickers):
    return _pd.concat([Fundamentals(t) for t in tickers], ignore_index=True)


def SaveToCSV(df, fname):
    df.to_csv(fname, index=False)
    print(f"Saved {len(df)} rows → {fname}")


if __name__ == "__main__":
    raw1 = ['aapl', 'amzn', 'meta', 'gm', 'f', 'uber' , 'amd', 'rdfn', 'rddt']
    raw = [
        "NET","CRS","ORLY","DOCN","TEM","VST","CVS","TER","RIVN","NTGR",
        "SPOT","UBER","SNOW","ACHR","ASAN","RKLB","DPZ","LYFT","NEGG",
        "MSFT","AAPL","ARM","AMZN","META","NFLX","NVDA","GOOG","RDFN",
        "MDB","Z","INTC","AMD","AFRM","COIN","TSLA","HOOD","CAVA",
        "F","GM","SMR","U","RDDT","SHOP","SOFI"
    ]
    tickers = [f"{t}.US" for t in raw1]
    df1 = FundamentalsBuilder(tickers)
    SaveToCSV(df1, "fundamentals_eod7.csv")


Saved 9 rows → fundamentals_eod7.csv


C:\Users\Daniel2\AppData\Local\Temp\ipykernel_21556\572806393.py:197: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return _pd.concat([Fundamentals(t) for t in tickers], ignore_index=True)


In [3]:
display(df1)

,Ticker,Current Price,All-Time High,ATH %Chg,6-Month High,Recent-High %Chg,5-Year High,Current P/E,Avg 5-Year P/E,Fair Value (TTM),Fair Value %Chg,Avg P/E × Fwd EPS,"MACD (12,26,9)",MACD Signal,RSI (1 yr)
0,NET,118.5250,217.2500,-0.454430,176.5000,-0.328470,217.2500,-526.086957,-146.375201,33.666296,-0.715956,-151.454421,-1.9686,-4.4208,53.2654
1,CRS,197.1750,212.0634,-0.070207,212.0634,-0.070207,212.0634,28.187000,16.940886,119.602657,-0.393419,150.780664,1.7996,-2.3823,55.4233
2,ORLY,1372.1950,1441.8900,-0.048336,1441.8900,-0.048336,1441.8900,33.006600,49.164518,2006.403988,0.462186,2429.336840,1.9714,7.7042,54.2471
3,DOCN,30.4900,130.2600,-0.765930,46.6900,-0.346969,130.2600,34.550600,171.496554,152.631933,4.005967,377.189521,-1.9394,-2.4629,49.1007
4,TEM,53.0900,89.4400,-0.406418,89.4400,-0.406418,89.4400,-8.491172,NaN,NaN,NaN,NaN,-0.4956,-2.1173,NaN
5,VST,127.0250,191.5597,-0.336891,191.5597,-0.336891,191.5597,18.091400,5.238299,36.668092,-0.711332,42.271501,-1.5473,-4.0621,52.7709
6,CVS,64.9900,99.0241,-0.343695,69.4639,-0.064406,99.0241,17.847000,9.301538,34.043631,-0.476171,64.093181,0.1815,0.5888,50.8630
7,TER,76.2100,166.2267,-0.541530,139.8511,-0.455063,166.2267,23.228900,30.797213,102.246748,0.341645,151.626999,-4.5514,-6.0224,47.4724
8,RIVN,12.8652,172.0100,-0.925207,16.4900,-0.219818,172.0100,-2.690832,-4.317334,20.248297,0.573881,12.582007,0.0292,-0.0593,50.0317
9,NTGR,24.4600,48.2170,-0.492710,30.9100,-0.208670,45.2300,58.952400,-141.751100,-59.535462,-3.433993,25.515198,-0.3667,-0.6480,52.9020
